In [ ]:
import numpy as np
from utils import *
from student_utils_sp18 import *
import networkx as nx
%matplotlib inline
print("networkx version: {}".format(nx.__version__))

In [ ]:
# reading a file and get graph
lines = read_file("data/sample_input.txt")

num_kingdoms, kingdom_names, start_kingdom, adj_matrix = data_parser(lines)

# map kingdom names to indices
name2index = {}
index2name = {}

count = 1
for name in kingdom_names:
    name2index[name] = count
    index2name[count] = name
    count += 1

G = adjacency_matrix_to_graph(adj_matrix)

G

In [ ]:
# initialize colors for each edge

color = {}

for node in G.nodes():
        color[node] = "white"
        
        
conquering_cost = {}
for node in G.nodes():
    conquering_cost[node] = adj_matrix[node][node]
    
nx.set_node_attributes(G, conquering_cost, "conquering_cost")
nx.set_node_attributes(G, color, "color")
# Example use G.node[1]['color'] gets the color of that node

In [ ]:
# calculate the heuristic for all nodes on the graph

def conquer_gain(G, node):
    """Given a Graph and a node it calculates the value of the heuristic for the node"""
    """conquering_cost_neighbors/conquering_cost_current_node larger is better"""
    
    if G.node[node]['color'] =="black": # do not calculate the heuristic for black nodes
        return 0
    else:
        neighbors = nx.neighbors(G, node)
        
        neighbors_conquering_cost = 0
        for n in neighbors:
            neighbors_conquering_cost += G.node[n]['conquering_cost']
        
        return neighbors_conquering_cost/G.node[node]['conquering_cost']

In [ ]:
# update the colors in the graph

def update_graph(G, node):
    G.node[node]['color'] = "black"
    neighbors = nx.neighbors(G, node)
    
    for n in neighbors:
        if G.node[n]['color'] == "white":
            G.node[n]['color'] = "grey"
    return

In [ ]:
# find the the nodes that we need to visit based on heuristic

def choose_node(G):
    # calculate heuristic for everyone
    heuristic_value = []

    for node in G.nodes():
        heuristic_value.append(conquer_gain(G, node))

    chosen_node = np.argmax(heuristic_value)
    return chosen_node

def all_conquered(G):
    lst = [l[1] for l in nx.get_node_attributes(G, "color").items()]
    
    return not "white" in lst

In [ ]:
# Identify components

nodes_to_visit = []

while not all_conquered(G):
    next_node = choose_node(G)
    nodes_to_visit.append(next_node)
    update_graph(G, next_node)

In [ ]:
# another way to identify components using an approximation to the 
# min weight dominating set

from networkx.algorithms.approximation import *

min_weighted_dominating_set(G, "conquering_cost")

In [ ]:
# Get the Steiner Tree based on the nodes you have identofied
from networkx.algorithms.traversal import *

# GET THE FULL WALK

start_index = name2index[start_kingdom]
ST = steinertree.steiner_tree(G, nodes_to_visit+[start_index])

In [ ]:
# recover the order of traversal of the tree

def find_traversal(tree, start_index):
    """Recovers the traversal of the Steiner tree using DFS (the full walk tho)"""
    
    vertex_order = list(dfs_preorder_nodes(tree, start_index))+[start_index]
    current_node = start_index
    full_walk = [start_index]

    for next_node in vertex_order[1:]:
        #print(current_node, next_node)
        path = list(nx.all_simple_paths(tree, current_node, next_node))[0]
        full_walk.extend(path[1:])
        current_node = next_node
    return full_walk

In [ ]:
traversal = find_traversal(ST, start_index)
conquered = dict(zip(nodes_to_visit, [False]*len(nodes_to_visit)))
to_conquer = []

for n in traversal:
    if n in nodes_to_visit and conquered[n] == False:
        to_conquer.append(n)
        conquered[n]=True
        
print("Traversal: {}".format(traversal))
print("To conquer: {}".format(to_conquer))
ccost = sum(G.node[i]['conquering_cost'] for i in to_conquer)
tcost = sum(G.get_edge_data(traversal[i], traversal[i+1])['weight'] for i in range(len(traversal)-1))
print("Cost of conquering: {}".format(ccost))
print("Cost of travelling: {}".format(tcost))

# Test run with Sue's input 1 file

In [ ]:
# reading a file and get graph
lines = read_file("data/input1.txt")

num_kingdoms, kingdom_names, start_kingdom, adj_matrix = data_parser(lines)

# map kingdom names to indices
name2index = {}
index2name = {}

count = 1
for name in kingdom_names:
    name2index[name] = count
    index2name[count] = name
    count += 1


G = adjacency_matrix_to_graph(adj_matrix)
pos = nx.spring_layout(G)
is_metric(G)

labels = nx.get_edge_attributes(G,'weight')
plt.figure(3,figsize=(8,8)) 
nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
nx.draw(G, pos, with_labels=True)

In [ ]:
def solver(G, start_index):
    # set the colors
    color = {}

    for node in G.nodes():
            color[node] = "white"


    conquering_cost = {}
    for node in G.nodes():
        conquering_cost[node] = adj_matrix[node][node]

    nx.set_node_attributes(G, conquering_cost, "conquering_cost")
    nx.set_node_attributes(G, color, "color")
    
    
    # find nodes to visit
    
    nodes_to_visit = []

    while not all_conquered(G):
        next_node = choose_node(G)
        nodes_to_visit.append(next_node)
        update_graph(G, next_node)
    
    # Handle the edge case from below
    if nodes_to_visit[0]==start_index:
        return [start_index], [start_index]
    
    # get Steiner tree
    # CURRENTLY HAS THE BUG THAT THE STEINER TREE DOES NOT WORK WHEN YOU ONLY HAVE 1 TARGET VERTEX
    ST = steinertree.steiner_tree(G, set(nodes_to_visit+[start_index]))

    # Find the traversal based on the Steiner tree
    traversal = find_traversal(ST, start_index)
    conquered = dict(zip(nodes_to_visit, [False]*len(nodes_to_visit)))
    to_conquer = []

    for n in traversal:
        if n in nodes_to_visit and conquered[n] == False:
            to_conquer.append(n)
            conquered[n]=True
            
    return traversal, to_conquer

In [ ]:
solver(G, 2)

In [ ]:
for i in range(7):
    print("Conquering cost {}: {}".format(i, G.node[i]['conquering_cost']))

In [ ]:
plt.figure(3,figsize=(8,8)) 
nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
nx.draw(G, pos, with_labels=True)

In [ ]:
# testing tree
# tree = nx.Graph()
# tree.add_node(1)
# tree.add_node(2)
# tree.add_node(3)
# tree.add_node(4)

# tree.add_edge(1,2)
# tree.add_edge(1,3)
# tree.add_edge(2,4)
# tree.add_edge(2,5)
# tree.add_edge(5, 6)

# nx.draw(tree, with_labels=True)